In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/lecture/"
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['movies.csv',
 'ratings.csv',
 'tags.csv',
 'links.csv',
 'movie_user.gsheet',
 '제목 없는 문서.gdoc',
 '협업 필터링 - Item-based.ipynb',
 'movies_refined.csv',
 'ratings_refined.csv',
 '무비렌즈 데이터 전처리.ipynb',
 '협업 필터링 - User-based.ipynb',
 '협업_필터링_Item_based_ipynb의_사본.ipynb',
 'Jacquard.csv',
 'Jacquard(mean).csv',
 'user_means(x).ipynb',
 'user_means(x).ipynb의 사본',
 'mode.ipynb',
 'zero.ipynb',
 'euclidean_similarity_movies.csv',
 'euclidean_similarity.ipynb',
 'movie_user.csv',
 'Untitled0.ipynb',
 '협업 필터링 - User-based with dot production.ipynb',
 'Untitled0(x).ipynb',
 'user_based_cf_prediction.csv',
 'euclidean_mean.ipynb',
 'euclidean_mode.ipynb의 사본']

In [3]:
import pandas as pd
import numpy as np

In [4]:
ratings = pd.read_csv(path + "ratings_refined.csv", usecols=['userId', 'movieId', 'rating'])

In [5]:
movies = pd.read_csv(path + "movies_refined.csv", usecols=['movieId', 'title'])

In [6]:
df = pd.merge(ratings, movies, on='movieId', how='left')

In [7]:
df.columns[df.isna().any()].tolist()

[]

In [8]:
df[df['title'].isnull()]

,userId,movieId,rating,title


In [9]:
movie_user = df.pivot_table(values='rating', index='title', columns='userId')
movie_user.to_csv('movie_user.csv')
movie_user

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN
xXx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,2.0
xXx: State of the Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5


In [10]:
movie_user.shape

(9413, 610)

In [11]:
from sklearn.metrics.pairwise import euclidean_distances

# 결측값을 평균 값으로 대체
movie_user_tmp = movie_user.copy()

# 각 열의 최빈값 계산 및 결측값 대체
for column in movie_user_tmp.columns:
    mode_val = movie_user_tmp[column].mode()[0]
    movie_user_tmp[column].fillna(mode_val, inplace=True)

# 유사도 행렬 계산
euclidean_similarity_matrix = 1 / (1 + euclidean_distances(movie_user_tmp, movie_user_tmp))

# 유사도 행렬 출력
print("Similarity Matrix:")
print(euclidean_similarity_matrix)

Similarity Matrix:
[[1.         0.66666667 0.53589838 ... 0.2        0.13997559 0.24744015]
 [0.66666667 1.         0.58578644 ... 0.21239994 0.14037601 0.25      ]
 [0.53589838 0.58578644 1.         ... 0.20942771 0.13957876 0.24496553]
 ...
 [0.2        0.21239994 0.20942771 ... 1.         0.12256538 0.17331936]
 [0.13997559 0.14037601 0.13957876 ... 0.12256538 1.         0.12789211]
 [0.24744015 0.25       0.24496553 ... 0.17331936 0.12789211 1.        ]]


In [12]:
euclidean_similarity_matrix

array([[1.        , 0.66666667, 0.53589838, ..., 0.2       , 0.13997559,
        0.24744015],
       [0.66666667, 1.        , 0.58578644, ..., 0.21239994, 0.14037601,
        0.25      ],
       [0.53589838, 0.58578644, 1.        , ..., 0.20942771, 0.13957876,
        0.24496553],
       ...,
       [0.2       , 0.21239994, 0.20942771, ..., 1.        , 0.12256538,
        0.17331936],
       [0.13997559, 0.14037601, 0.13957876, ..., 0.12256538, 1.        ,
        0.12789211],
       [0.24744015, 0.25      , 0.24496553, ..., 0.17331936, 0.12789211,
        1.        ]])

In [13]:
movie_titles = movie_user.index
movie_titles

Index([''71', ''Hellboy': The Seeds of Creation', ''Round Midnight',
       ''Salem's Lot', ''Til There Was You', ''Tis the Season for Love',
       ''burbs, The', ''night Mother', '(500) Days of Summer',
       '*batteries not included',
       ...
       'Zulu', '[REC]', '[REC]²', '[REC]³ 3 Génesis',
       'anohana: The Flower We Saw That Day - The Movie', 'eXistenZ', 'xXx',
       'xXx: State of the Union', '¡Three Amigos!',
       'À nous la liberté (Freedom for Us)'],
      dtype='object', name='title', length=9413)

In [14]:
movie_similarity = pd.DataFrame(euclidean_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity

(9413, 9413)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.666667,0.535898,0.472136,0.400000,0.326632,0.166667,0.585786,0.123360,0.348331,...,0.252650,0.281729,0.585786,0.500000,0.387426,0.180011,0.121276,0.200000,0.139976,0.247440
'Hellboy': The Seeds of Creation,0.666667,1.000000,0.585786,0.500000,0.414214,0.333333,0.167366,0.666667,0.123629,0.356789,...,0.255397,0.277926,0.666667,0.666667,0.400000,0.180907,0.123094,0.212400,0.140376,0.250000
'Round Midnight,0.535898,0.585786,1.000000,0.449490,0.387426,0.320377,0.167366,0.535898,0.123094,0.340542,...,0.250000,0.270813,0.535898,0.535898,0.376179,0.179129,0.122565,0.209428,0.139579,0.244966
'Salem's Lot,0.472136,0.500000,0.449490,1.000000,0.500000,0.309017,0.164624,0.472136,0.122565,0.326632,...,0.244966,0.264279,0.472136,0.472136,0.356789,0.177405,0.122044,0.206587,0.138796,0.240253
'Til There Was You,0.400000,0.414214,0.387426,0.500000,1.000000,0.289898,0.162021,0.400000,0.121530,0.303832,...,0.235828,0.252650,0.400000,0.400000,0.326632,0.174112,0.121023,0.201263,0.137270,0.231662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0.180011,0.180907,0.179129,0.177405,0.174112,0.168074,0.130684,0.178260,0.101858,0.170259,...,0.155487,0.160768,0.180011,0.180011,0.173319,1.000000,0.108688,0.145935,0.112689,0.155487
xXx,0.121276,0.123094,0.122565,0.122044,0.121023,0.119057,0.105756,0.122829,0.090909,0.121786,...,0.122565,0.111111,0.122829,0.124446,0.120771,0.108688,1.000000,0.132320,0.097209,0.114550
xXx: State of the Union,0.200000,0.212400,0.209428,0.206587,0.201263,0.191825,0.138796,0.210897,0.109601,0.195194,...,0.174917,0.160153,0.210897,0.224009,0.200000,0.145935,0.132320,1.000000,0.122565,0.173319


In [15]:
movie_similarity = pd.DataFrame(euclidean_similarity_matrix,
                                index=movie_titles, columns=movie_titles)

# 샘플 영화에 대한 유사한 영화 찾기
sample_movie = "'71"  # 여기에 샘플 영화 제목을 넣어주세요
similar_movies = movie_similarity[sample_movie].sort_values(ascending=False)[1:10]

# 결과 출력
print(similar_movies)

title
Resolution                 1.0
House of the Devil, The    1.0
Perfect Getaway, A         1.0
Guest, The                 1.0
Lifted                     1.0
Stake Land                 1.0
Room for Romeo Brass, A    1.0
Masked Avengers            1.0
Brothers Bloom, The        1.0
Name: '71, dtype: float64


In [16]:
from tqdm.notebook import tqdm

# 예측을 위한 함수
def modeling(similarity_matrix, data):
    # 데이터 프레임 초기화 및 사용자 및 영화 정보 추출
    df_pred_all = pd.DataFrame()
    user_ids = sorted(data['userId'].unique())
    all_movies = similarity_matrix.index
    n_movies = len(all_movies)

    for user_id in tqdm(user_ids):
        idx = data[data['userId'] == user_id].index

        # 현재 사용자가 이미 시청한 영화들에 대한 유사도 행렬 추출
        watched_movie_ids = data.loc[idx, 'title'].tolist()
        sub_sim_mat = similarity_matrix.loc[watched_movie_ids]
        sub_sim_mat = sub_sim_mat.T.to_numpy()

        # 유사도에 대한 정규화 항 계산
        sim_N = np.sum(sub_sim_mat, axis=1) + 1  # 1을 더하는 이유 => 유사도의 합이 0이 되는 경우를 방지

        # 현재 사용자가 시청한 영화에 대한 평점 가져오기
        watched_movie_ratings = data.loc[idx, 'rating']
        watched_movie_ratings = np.array(watched_movie_ratings.tolist()).reshape(-1, 1)

        # 가중 평균을 사용하여 평점 예측
        pred_y = np.matmul(sub_sim_mat, watched_movie_ratings).flatten() / sim_N

        user_id_list = [user_id] * n_movies
        cur_pred = pd.DataFrame(zip(user_id_list, all_movies, pred_y),
                                columns=['userId', 'title', 'pred_rating'])

        # 결과 기록
        df_pred_all = pd.concat([df_pred_all, cur_pred], axis=0)

    return df_pred_all

# 예시 사용
df_pred_all = modeling(movie_similarity, df)


  0%|          | 0/610 [00:00<?, ?it/s]

In [17]:
df_pred_all

,userId,title,pred_rating
0,1,'71,4.255953
1,1,'Hellboy': The Seeds of Creation,4.257282
2,1,'Round Midnight,4.255562
3,1,'Salem's Lot,4.252387
4,1,'Til There Was You,4.248214
...,...,...,...
9408,610,eXistenZ,3.627744
9409,610,xXx,3.596694
9410,610,xXx: State of the Union,3.563285
9411,610,¡Three Amigos!,3.628471


In [18]:
def get_unwatched_movies(user_id, watched_movies, all_movies):
    unwatched_movies = set(all_movies) - set(watched_movies)
    num_unwatched_movies = len(unwatched_movies)
    return list(unwatched_movies), num_unwatched_movies

# 예시 사용
user_id_to_check = 1  # 사용자 ID를 적절히 설정
watched_movies_by_user = df[df['userId'] == user_id_to_check]['title'].tolist()
all_movies_list = df['title'].unique().tolist()

unwatched_movies_list, num_unwatched_movies = get_unwatched_movies(user_id_to_check, watched_movies_by_user, all_movies_list)

# 처음 10개의 영화만 출력
print(f"총 {num_unwatched_movies}개의 영화가 남아있습니다.")
print(f"사용자 {user_id_to_check}가 아직 보지 않은 영화 목록:")
for movie in unwatched_movies_list[:10]:
    print(movie)

총 9184개의 영화가 남아있습니다.
사용자 1가 아직 보지 않은 영화 목록:
Judd Apatow: The Return
Van Helsing
Snowman, The
Trip to the Moon, A (Voyage dans la lune, Le)
Down by Law
Ladykillers, The
Billy Jack
Final Destination
Tangled
Parenthood


In [19]:
def recommend_top_movies_for_user(user_id, df_user, df_pred, top_n=10):
    # 사용자가 이미 본 영화 목록 가져오기
    watched_movies_by_user = df_user[df_user['userId'] == user_id]['title'].tolist()

    # 모든 영화 목록 가져오기
    all_movies_list = df_user['title'].unique().tolist()

    # 사용자가 아직 보지 않은 영화 목록 가져오기
    unwatched_movies_list, _ = get_unwatched_movies(user_id, watched_movies_by_user, all_movies_list)

    # 사용자가 아직 보지 않은 영화 중에서 pred_rating이 높은 상위 N개 영화 추천
    user_pred_data = df_pred[df_pred['userId'] == user_id]
    unwatched_movies_pred = user_pred_data[user_pred_data['title'].isin(unwatched_movies_list)]

    # 평점이 높은 상위 N개 영화 추천
    top_movies = unwatched_movies_pred.sort_values(by='pred_rating', ascending=False).head(top_n)

    return top_movies[['title', 'pred_rating']].values.tolist()

# 사용자가 아직 보지 않은 영화 중에서 pred_rating이 높은 상위 10개 추천 받기
user_id_to_recommend = 1  # 사용자 ID를 적절히 설정
recommended_movies = recommend_top_movies_for_user(user_id_to_recommend, df, df_pred_all, top_n=10)

# 결과 출력
print(f"사용자 {user_id_to_recommend}에게 추천되는 영화:")
for movie, pred_rating in recommended_movies:
    print(f"{movie} - 예측 평점: {pred_rating}")

사용자 1에게 추천되는 영화:
Cats Don't Dance - 예측 평점: 4.258557815437268
Soft Fruit - 예측 평점: 4.258557815437268
The Square - 예측 평점: 4.257985588421916
Ararat - 예측 평점: 4.257985588421916
A Most Wanted Man - 예측 평점: 4.257985588421916
Proof - 예측 평점: 4.257985588421916
Copenhagen - 예측 평점: 4.257985588421916
Shootist, The - 예측 평점: 4.257736866346725
Detour - 예측 평점: 4.257736866346725
Dad's Army - 예측 평점: 4.25771557070966


In [20]:
def recommend_top_movies_for_user_input(user_id, df_user, df_pred, top_n=10):
    # 사용자가 이미 본 영화 목록 가져오기
    watched_movies_by_user = df_user[df_user['userId'] == user_id]['title'].tolist()

    # 모든 영화 목록 가져오기
    all_movies_list = df_user['title'].unique().tolist()

    # 사용자가 아직 보지 않은 영화 목록 가져오기
    unwatched_movies_list, _ = get_unwatched_movies(user_id, watched_movies_by_user, all_movies_list)

    # 사용자가 아직 보지 않은 영화 중에서 pred_rating이 높은 상위 N개 영화 추천
    user_pred_data = df_pred[df_pred['userId'] == user_id]
    unwatched_movies_pred = user_pred_data[user_pred_data['title'].isin(unwatched_movies_list)]

    # 평점이 높은 상위 N개 영화 추천
    top_movies = unwatched_movies_pred.sort_values(by='pred_rating', ascending=False).head(top_n)

    return top_movies[['title', 'pred_rating']].values.tolist()

# 사용자로부터 입력 받기
user_id_to_recommend = int(input("사용자 ID를 입력하세요: "))

# 사용자가 아직 보지 않은 영화 중에서 pred_rating이 높은 상위 10개 추천 받기
recommended_movies = recommend_top_movies_for_user_input(user_id_to_recommend, df, df_pred_all, top_n=10)

# 결과 출력
print(f"사용자 {user_id_to_recommend}에게 추천되는 영화:")
for movie, pred_rating in recommended_movies:
    print(f"{movie} - 예측 평점: {pred_rating}")

사용자 ID를 입력하세요: 77
사용자 77에게 추천되는 영화:
Star Trek - 예측 평점: 2.7469983292028224
Creep - 예측 평점: 2.7416258250826724
Ninja Assassin - 예측 평점: 2.741235579306341
Mother! - 예측 평점: 2.741181015915797
5 Centimeters per Second (Byôsoku 5 senchimêtoru) - 예측 평점: 2.740484363482875
Dear Zachary: A Letter to a Son About His Father - 예측 평점: 2.740288911359237
Frances Ha - 예측 평점: 2.739788080698675
Only Yesterday (Omohide poro poro) - 예측 평점: 2.739788080698675
Wadjda - 예측 평점: 2.739788080698675
Pawn Sacrifice - 예측 평점: 2.739788080698675


In [21]:
from sklearn.model_selection import train_test_split

_, test_data = train_test_split(df[['userId', 'title', 'rating']],
                test_size=20000, random_state=1234, stratify=df['userId'])
test_data

,userId,title,rating
72039,464,Escape from L.A.,3.0
90528,590,Carlito's Way,4.0
20360,135,"Nightmare Before Christmas, The",4.0
42826,288,"Naked Gun 2 1/2: The Smell of Fear, The",4.0
37923,258,Guardians of the Galaxy 2,3.5
...,...,...,...
94958,599,Thor: Ragnarok,3.0
63609,414,Revenge of the Nerds II: Nerds in Paradise,2.0
68727,448,"Wild Bunch, The",5.0
38371,263,Gosford Park,4.5


In [22]:
test_data = pd.merge(test_data, df_pred_all, on=['userId', 'title'], how='left')
test_data

,userId,title,rating,pred_rating
0,464,Escape from L.A.,3.0,3.318377
1,590,Carlito's Way,4.0,3.331165
2,135,"Nightmare Before Christmas, The",4.0,3.465123
3,288,"Naked Gun 2 1/2: The Smell of Fear, The",4.0,3.088110
4,258,Guardians of the Galaxy 2,3.5,2.985422
...,...,...,...,...
19995,599,Thor: Ragnarok,3.0,2.598044
19996,414,Revenge of the Nerds II: Nerds in Paradise,2.0,3.289014
19997,448,"Wild Bunch, The",5.0,2.818234
19998,263,Gosford Park,4.5,3.575763


In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

true_y = np.array(test_data['rating'])
pred_y = np.array(test_data['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 2))}")
print(f"MSE  : {str(round(mse, 2))}")
print(f"RMSE : {str(round(rmse, 2))}")

MAE  : 0.74
MSE  : 0.86
RMSE : 0.93


In [24]:
# 모델이 추천한 영화 개수
n_recommends = sum(1 * (pred_y > 4.0))
n_recommends

867

In [25]:
n_movies = df['title'].nunique()

In [26]:
# Coverage
n_recommends / n_movies

0.09210666100074365

In [27]:
def get_precision(true_y, pred_y, threshold):
    trues = 1 * (true_y >= threshold)
    n_trues = sum(trues)

    pred_trues = 1 * (pred_y >= threshold)

    true_positive = sum(trues + pred_trues == 2)

    precision = true_positive / n_trues

    return precision

In [28]:
get_precision(true_y, pred_y, 3)

0.8324394303680415

In [29]:
df_pred_all.to_csv(path + "user_based_cf_prediction.csv", index=False)